In [1]:
import numpy as np
import pandas as pd

In [2]:
import pandas as pd
import os
import json
from datetime import datetime
from tqdm import tqdm
import math
from dateutil import parser
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
def get_subfolder_paths(top_folder_path):
    paths = []
    sub_folders = next(os.walk(top_folder_path))[1]
    print("Top Folder Path:", top_folder_path)
    print("Sub Folders Found:", sub_folders)
    for sub_folder in sub_folders:
        for folder in ['P', 'R']:
            folder_path = os.path.join(top_folder_path, sub_folder, folder)
            if os.path.exists(folder_path):
                paths.append(folder_path)
    normalized_paths = [path.replace('\\', '/') for path in paths]
    return  normalized_paths

In [4]:
def get_all_files_path_under(folder_path):
    
    all_jsons = []

    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            all_jsons.append( os.path.normpath (file_path) )
    return all_jsons


In [5]:
def get_distance(shooterTeamId,homeId,homeSide,x,y):
    if shooterTeamId == homeId:
        if homeSide == 'right':
            return  math.sqrt((x +100)**2 + (y)**2)
        if homeSide == 'left':
            return  math.sqrt((x -100)**2 + (y)**2)
    if shooterTeamId != homeId:
        if homeSide == 'right':
            return  math.sqrt((x - 100)**2 + (y)**2)
        if homeSide == 'left':
            return  math.sqrt((x + 100)**2 + (y)**2)

In [6]:
def get_angle(shooterTeamId,homeId,homeSide,x,y):
    if shooterTeamId == homeId:
        if homeSide == 'right':
            a = abs(y)
            b = math.sqrt((x + 100)**2 + (y)**2)
            c = abs(x + 100)
            return math.degrees(math.acos((b**2 + c**2 - a**2)/(2*b*c)))
        if homeSide == 'left':
            a = abs(y)
            b = math.sqrt((x -100)**2 + (y)**2)
            c = abs(x - 100)
            return math.degrees(math.acos((b**2 + c**2 - a**2)/(2*b*c)))
    if shooterTeamId != homeId:
        if homeSide == 'right':
            a = abs(y)
            b = math.sqrt((x -100)**2 + (y)**2)
            c = abs(x - 100)
            return math.degrees(math.acos((b**2 + c**2 - a**2)/(2*b*c)))
        if homeSide == 'left':
            a = abs(y)
            b = math.sqrt((x + 100)**2 + (y)**2)
            c = abs(x + 100)
            return math.degrees(math.acos((b**2 + c**2 - a**2)/(2*b*c)))



In [7]:
def create_csv(path):
    data_to_tidy = get_all_files_path_under(path)

    data_list = []

    for j in tqdm(range(len(data_to_tidy))):
        
        json_file = data_to_tidy[j]
     
        with open(json_file) as json_file:
            data = json.load(json_file)


        homeId =  data.get('homeTeam').get('id')
        awayId =  data.get('awayTeam').get('id')

        plays = data.get("plays")

        ID = data.get('id')

        plays = data.get('plays')

        home_score = 0
        away_score = 0
        for i in range(len(plays)):
                    
            if plays[i]['typeDescKey'] in ["missed-shot", "goal" ,'shot-on-goal']:
                details =  plays[i].get('details')
                if 'xCoord' in details.keys():
                    homeSide = plays[i]['homeTeamDefendingSide']
                    shooterTeamId = details['eventOwnerTeamId']
                    homeSide = plays[i]['homeTeamDefendingSide']

                    x = details['xCoord']
                    y = details['yCoord']
               
   
                    distance = get_distance(shooterTeamId,homeId,homeSide,x,y)
                    angle = get_angle(shooterTeamId,homeId,homeSide,x,y)
                    isGoal = 0
                    emptyNet = 0
                    if  plays[i]['typeDescKey'] ==  "goal":
                        isGoal = 1
                        if shooterTeamId == homeId:
                            home_score += 1
                            if int(plays[i].get('situationCode')[0]) == 0:
                                emptyNet= 1
                        if shooterTeamId == awayId:
                            away_score += 1
                            if int(plays[i].get('situationCode')[3]) == 0:
                                emptyNet = 1

                    new_row = {'gameId':ID,'distance':distance, 'angle':angle, 'emptyNet':emptyNet,'isGoal':isGoal,'home_score':home_score,
                               'away_score':away_score}
                    data_list.append(new_row)
        df = pd.DataFrame(data_list)        
        path =os.path.normpath(path) 
        path.split('\\')[-2:]
        csv_name = path.split('\\')[-2]+'_'+ path.split('\\')[-1]+'_clean'+'.csv'
        folder = './ift6758/ift6758/data/data_tidy/'+csv_name

        df.to_csv(folder, encoding='utf-8', index=False)



         


In [8]:
folder_path = './ift6758/ift6758/data/data_tidy'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f"Folder created at {folder_path}")
else:
    print(f"Folder already exists at {folder_path}")

Folder already exists at ./ift6758/ift6758/data/data_tidy


In [9]:
json_path = './ift6758/ift6758/data/data_json/'
paths = get_subfolder_paths(json_path)

Top Folder Path: ./ift6758/ift6758/data/data_json/
Sub Folders Found: ['2019-2020', '2020-2021', '2021-2022', '2022-2023']


In [10]:
for path in paths:
    create_csv(path)

100%|██████████████████████████████████████████████████████████████████████████████| 1312/1312 [05:08<00:00,  4.26it/s]


In [11]:
data = pd.read_csv('./ift6758/ift6758/data/data_tidy/2019-2020_P_clean.csv')

In [14]:
data

,gameId,distance,angle,emptyNet,isGoal,home_score,away_score
0,2019030111,36.235342,27.979474,0,0,0,0
1,2019030111,32.649655,49.969741,0,0,0,0
2,2019030111,24.351591,19.179008,0,0,0,0
3,2019030111,65.863495,30.068583,0,0,0,0
4,2019030111,44.283180,6.483074,0,0,0,0
...,...,...,...,...,...,...,...
7482,2019030416,27.073973,4.236395,0,0,0,2
7483,2019030416,69.720872,8.246321,0,0,0,2
7484,2019030416,17.464249,23.629378,0,0,0,2
7485,2019030416,23.000000,0.000000,0,0,0,2
